In [1]:
import py_entitymatching as em #Import mangellan library
import math

In [2]:
# This function was used for approximate verification of tagged_data, since we noticed that, 
# In case of a match, the phone numbers matched in most of the cases. 

# Matches phone numbers - Matches only the digits and drops other characters. 
# Handles NaN values.
# If both the fields to be matches are NaN, the result is passed as True to bypass this test for that case. 

def phone_match(str1,str2):
    if type(str1) is float and type(str2) is float:
        if math.isnan(str1) and math.isnan(str2):
            return True
    elif type(str1) is float:
        if math.isnan(str1):
            return False
    elif type(str2) is float:
        if math.isnan(str2):
            return False
    else:
        stra = ""
        strb = ""
        for ch in str1:
            if ch.isdigit():
                stra += ch
        for ch in str2:
            if ch.isdigit():
                strb += ch
        if stra == strb:
            return True
        else:
            return False

In [4]:
# Read the CSV files
yelp = em.read_csv_metadata('./yelp.csv', key='id')
zomato = em.read_csv_metadata('./zomato.csv', key='id')

In [7]:
#Printing first few tuples of Zomato table
yelp.head(1)

,id,Name,Phone,Zipcode,State,City,Address,Delivery,Takeout,Outdoor_seating
0,1,Nini’s Deli,(312) 243-9822,60642.0,IL,Chicago,543 N Noble St,0,1,1


In [8]:
#Printing first few tuples of Zomato table
zomato.head(1)

,id,Name,Phone,Zipcode,State,City,Address,Delivery,Takeout,Outdoor_seating
0,1,Alinea,(312) 867-0110,60614.0,IL,Chicago,1723 N. Halsted Street,0,0,0


In [9]:
# Display the number of tuples in the input dataset 
print len(yelp), len(zomato)

3075 3478


In [24]:
# Downsample the datasets 
yelp_subset, zomato_subset = em.down_sample(yelp, zomato, size=2500, y_param=1, show_progress=False)

In [25]:
# Display the number of tuples in the sampled datasets
len(yelp_subset), len(zomato_subset)

(849, 2500)

### Carry out Overlapping Blocking on Address

In [26]:
# Instantiate overlap blocker object
ob = em.OverlapBlocker()

# Get the list of attributes of yelp table
yelp_attrs = []
for attr in yelp.keys():
    yelp_attrs.append(attr)

# Get the list of attributed of zomato table
zomato_attrs = []
for attr in zomato.keys():
    zomato_attrs.append(attr)

print "yelp attributes: ", yelp_attrs
print "\nzomato attributes: ", zomato_attrs


# Specify the tokenization to be 'word' level and set overlap_size to be 2.
C1 = ob.block_tables(yelp_subset, zomato_subset, 'Address', 'Address', q_val=3, word_level=False, overlap_size=3,
                     rem_stop_words=False,
                     l_output_attrs=yelp_attrs, r_output_attrs=zomato_attrs, show_progress=False)

yelp attributes:  ['id', 'Name', 'Phone', 'Zipcode', 'State', 'City', 'Address', 'Delivery', 'Takeout', 'Outdoor_seating']

zomato attributes:  ['id', 'Name', 'Phone', 'Zipcode', 'State', 'City', 'Address', 'Delivery', 'Takeout', 'Outdoor_seating']


In [27]:
C1.shape

(146597, 21)

In [28]:
C1.head()

,_id,ltable_id,rtable_id,ltable_Name,ltable_Phone,ltable_Zipcode,ltable_State,ltable_City,ltable_Address,ltable_Delivery,...,ltable_Outdoor_seating,rtable_Name,rtable_Phone,rtable_Zipcode,rtable_State,rtable_City,rtable_Address,rtable_Delivery,rtable_Takeout,rtable_Outdoor_seating
0,0,2392,1633,Chiso Restaurant,(206) 632-3430,98103.0,WA,Seattle,3520 Fremont Ave N,0,...,0,Tonga Room & Hurricane Bar,(415) 772-5278,94108.0,CA,San Francisco,Fairmont San Francisco Hotel,0,0,0
1,1,90,1633,MK Restaurant,(312) 482-9179,60610.0,IL,Chicago,868 N Franklin St,0,...,0,Tonga Room & Hurricane Bar,(415) 772-5278,94108.0,CA,San Francisco,Fairmont San Francisco Hotel,0,0,0
2,2,126,1633,The Capital Grille,(312) 337-9400,60611.0,IL,Chicago,633 N Saint Clair St,0,...,0,Tonga Room & Hurricane Bar,(415) 772-5278,94108.0,CA,San Francisco,Fairmont San Francisco Hotel,0,0,0
3,3,179,1633,Tru,(312) 202-0001,60611.0,IL,Chicago,676 N Saint Clair St,0,...,0,Tonga Room & Hurricane Bar,(415) 772-5278,94108.0,CA,San Francisco,Fairmont San Francisco Hotel,0,0,0
4,4,181,1633,BIG & Little's Restaurant,(773) 857-6677,60657.0,IL,Chicago,1034 W Belmont Ave,1,...,1,Tonga Room & Hurricane Bar,(415) 772-5278,94108.0,CA,San Francisco,Fairmont San Francisco Hotel,0,0,0


In [29]:
#Used to check the detected stop words for debugging
#ob.stop_words

## Blocking of name

In [30]:
# Specify the tokenization to be 'word' level and set overlap_size to be 1.
C2 = ob.block_candset(C1, 'Name', 'Name', word_level=False, q_val= 3, overlap_size=1, show_progress=False)

0%                          100%
[##############################] | ETA: 00:00:12 | ETA: 00:00:09 | ETA: 00:00:07 | ETA: 00:00:07 | ETA: 00:00:06 | ETA: 00:00:06 | ETA: 00:00:06 | ETA: 00:00:05 | ETA: 00:00:05 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:06


In [31]:
C2.shape

(33506, 21)

In [32]:
# Instantiate Attr. Equivalence Blocker
ab = em.AttrEquivalenceBlocker()
# Use block_tables to apply blocking over two input tables.
C3 = ab.block_candset(C2, l_block_attr='Zipcode', r_block_attr='Zipcode', show_progress=False)

In [40]:
# Number of tuples in dataset after blocking
len(C3)

1688

In [34]:
# Checking by matching phone numbers. Just a way of verifying the data.
# Not intending to get the exact match, just get a rough estimate.

count = 0
for r1,p1,r2,p2 in zip(C3['ltable_Name'],C3['ltable_Phone'],C3['rtable_Name'],C3['rtable_Phone']):
    #print p1,"\t",p2
    if phone_match(p1,p2):
        #print p1, p2
        count += 1
print count

399


In [35]:
# To find the attributes of C3
C3.columns

Index([u'_id', u'ltable_id', u'rtable_id', u'ltable_Name', u'ltable_Phone',
       u'ltable_Zipcode', u'ltable_State', u'ltable_City', u'ltable_Address',
       u'ltable_Delivery', u'ltable_Takeout', u'ltable_Outdoor_seating',
       u'rtable_Name', u'rtable_Phone', u'rtable_Zipcode', u'rtable_State',
       u'rtable_City', u'rtable_Address', u'rtable_Delivery',
       u'rtable_Takeout', u'rtable_Outdoor_seating'],
      dtype='object')

In [37]:
# Write C3 to csv for Manual Marking(Labelling)
C3.to_csv("blocked_dataset_2.csv") #Too many attributes. Difficult for labelling

In [38]:
#Downsampling the blocked table to be tagged
S = em.sample_table(C3, 1100)
S.shape

(1100, 21)

In [39]:
#Writing this table to disk
S.to_csv("to_be_tagged_2.csv")